# Bonds Portfolio Hedge

### Members

Marcos Cavalcanti - 1920533 

(And only me...)

## Install Libraries

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import datetime as dt

## Question 01

https://www.tesourodireto.com.br/mercado-de-titulos-publicos/rentabilidade-acumulada.htm

In [3]:
def LTN_price(rate,working_days):
    price = 1000 / (1 + rate)**(working_days / 252)
    return price

ltn_yield = 0.0614  #(6,14% a.a.)
working_days = 1209
print("LTN Price = %.4f"%(LTN_price(ltn_yield,working_days)))

LTN Price = 751.3495


##### I could only acquire the NTNF's price and infere its YTM (yield to maturity) !

https://maisretorno.com/titulos-publicos/ntn-f-01-01-2023-brstncntf147

In [4]:
def NTNF_price(rate,coupon_days,yield_=0.10):
        price = 0
        for working_days in coupon_days :
            1==1
            price += (1000 * ((1+yield_)**(1/2)-1))/ ((1 + rate)**(working_days / 252))
        price += 1000 / (1 + rate)**(working_days / 252)
        return price

rate = 0.0565 #(5,65% a.a.)
coupon_days = [77,204,328,455,580,706]
print("NTNF Price = %.4f"%(NTNF_price(rate,coupon_days)))

NTNF Price = 1126.4598


## Question 02

### Duration

Let $P \in \mathbb{R}_{\geq0}$ be the bond's price, $C_t\in \mathbb{R}_{\geq0}$ the payment at the instant $t \in \{1,...,T\}$, $y \in \mathbb{R}_{\geq0}$ the internal rate of discount given by the market and $T \in \mathbb{N}$ the number of periods. Then,

$P = \sum\limits_{t = 1}^{T} \frac{C_t}{(1+y)^{t}}\; \rightarrow\; \frac{dP}{dy} = -\frac{1}{1+y}\sum\limits_{t=1}^{T} \frac{t C_t}{(1+y)^{t}} \;\rightarrow\; D = -\frac{1}{P}\frac{dP}{dy} = \frac{1}{1+y}\sum\limits_{t=1}^{T} \frac{1}{P}\;\frac{t C_t}{(1+y)^t} = \frac{DM}{(1+y)}\;$, where

$D$ is named Modified Duration and $DM$, Macaulay Duration, such that

$DM = \frac{1}{P}\sum\limits_{t=1}^{T} \frac{t C_t}{(1+y)^t} $.

### Duration Functions

In [5]:
## Macaulay Duration   
def duration_macaulay(rate,coupon_days,yield_=0.10):
    duration = 0
    price = 0
    if len(coupon_days) == 1:
        duration = coupon_days[0]/252
    else:
        for working_days in coupon_days:
            price += (1000 * (((1+yield_)**(1/2))-1))/ ((1 + rate)**(working_days / 252))
            duration += (working_days/252) * (1000 * (((1+yield_)**(1/2))-1)) / ((1 + rate)**(working_days / 252))
        price += 1000 / (1 + rate)**(working_days / 252)
        duration += (working_days/252) * 1000 / (1 + rate)**(working_days / 252)
        duration = duration/price
    return duration


## Modified Duration
def duration_modified(rate,coupon_days,yield_=0.10):
    return duration_macaulay(rate,coupon_days,yield_) / (1+rate)


### LTN

#### Theoretical

In [6]:
ltn_yield = 0.0614 #(6,14% a.a.)
working_days = 1209
coupon_days = [working_days]

print("Theoretical Duration = %.4f years"%(duration_modified(ltn_yield,coupon_days)))

Theoretical Duration = 4.5201 years


#### Approximate

In [7]:
ltn_yield = 0.0614 #(6,14% a.a.)
working_days = 1209
coupon_days = [working_days]

delta = 0.0001 #(0,01%)

print("Aproximate Duration = %.4f years"% ((LTN_price(ltn_yield - delta,working_days) - LTN_price(ltn_yield + delta,working_days)) / (2*LTN_price(ltn_yield,working_days)*delta)))

Aproximate Duration = 4.5201 years


### NTN-F

#### Theoretical

In [8]:
rate = 0.0565 #(5,65% a.a.)
coupon_days = [77,204,328,455,580,706]

print("Theoretical Duration = %.4f years"%(duration_modified(rate,coupon_days)))

Theoretical Duration = 2.3606 years


#### Approximate

In [9]:
rate = 0.0565 #(5,65% a.a.)
coupon_days = [77,204,328,455,580,706]

delta = 0.0001 #(0,01%)

print("Aproximate Duration = %.4f years"% ((NTNF_price(rate - delta,coupon_days) - NTNF_price(rate + delta,coupon_days)) / (2*NTNF_price(rate,coupon_days)*delta)))

Aproximate Duration = 2.3606 years


### Portfolio Duration

Provided the individual duration from each bond in the portfolio, how could we hedge it?

Well, given the linearity, the portfolio's duration is given by a "present-value-proportional" average. Which is...

$D^{portfolio} = \frac{\sum\limits_{j = 1}^{k} P_j D_j}{\sum\limits_{j = 1}^{k} P_j}$

In [16]:
working_days = 1209
ltn_coupon_days = [working_days]
ntnf_coupon_days = [77,204,328,455,580,706]

D_portfolio = (LTN_price(ltn_yield,working_days)*duration_modified(rate,ltn_coupon_days) + NTNF_price(rate,coupon_days)*duration_modified(rate,ntnf_coupon_days)) / (LTN_price(ltn_yield,working_days)+NTNF_price(rate,coupon_days))
print("Portfolio Duration = %.4f years or %d days"%(D_portfolio,D_portfolio*252))

Portfolio Duration = 3.2330 years or 814 days


Therefore, the DI1J23 (770 days and 5.37%) was the future contract chosen for the job!

### Questions 03 / 05

In [17]:
DI_future = pd.read_csv("https://raw.githubusercontent.com/marcosaacavalcanti/QuantitativeFinance/main/Datasets/DI_dataset.csv",sep = ";")
DI_future["date"] = pd.to_datetime(DI_future["date"]).dt.strftime('%Y-%m-%d')
DI_future.set_index(["date"],inplace = True)
DI_future

,opening_rate,adjustment,DI
date,,,
2020-10-03,5.37,85179.51,0.000161
2020-11-03,5.30,85118.12,0.000161
2020-12-03,7.10,82195.24,0.000161
2020-03-13,6.36,84664.80,0.000161
2020-03-16,5.81,85153.99,0.000161
2020-03-17,5.93,86417.59,0.000161
2020-03-18,5.92,82779.56,0.000161
2020-03-19,7.02,82952.50,0.000142
2020-03-20,6.58,83060.78,0.000142


### Question 04

In [18]:
quantity_bonds = 10000
ltn_yield = 0.06143
working_days = 1209
rate = 0.0565 #(5,65% a.a.)
coupon_days = [77,204,328,455,580,706]

exposure = (LTN_price(ltn_yield,working_days) + NTNF_price(rate,coupon_days)) * quantity_bonds
quantity_contracts = round((exposure / 100000)/5) * 5
print("Quantity of future contracts: %d"%(quantity_contracts))

Quantity of future contracts: 190


### Question 06

In [19]:
DI_future["opening_price"] = np.nan
DI_future["transfers"] = np.nan
for i,dt in enumerate(DI_future.index):
    DI_future.loc[dt,["opening_price"]] = 100000 / (1+float(DI_future.loc[dt,["opening_rate"]])/100)**((770-i)/252)
    DI_future.loc[dt,["transfers"]] = (float(DI_future.loc[dt,["adjustment"]]) - ((1+ float(DI_future.loc[dt,["DI"]])) * float(DI_future.loc[dt,["opening_price"]])))*quantity_contracts
DI_future.loc[dt,["transfers"]] = 0
DI_future

,opening_rate,adjustment,DI,opening_price,transfers
date,,,,,
2020-10-03,5.37,85179.51,0.000161,85228.923963,-11995.805811
2020-11-03,5.30,85118.12,0.000161,85419.665673,-59906.665489
2020-12-03,7.10,82195.24,0.000161,81135.896887,198793.244346
2020-03-13,6.36,84664.80,0.000161,82888.850849,334894.768792
2020-03-16,5.81,85153.99,0.000161,84226.077292,173726.938897
2020-03-17,5.93,86417.59,0.000161,83955.576553,465214.353880
2020-03-18,5.92,82779.56,0.000161,83998.809750,-234226.975996
2020-03-19,7.02,82952.50,0.000142,81430.422729,286997.688766
2020-03-20,6.58,83060.78,0.000142,82473.401475,109376.787390


In [20]:
print("Final Balance: R$ %.2f"%(DI_future["transfers"].sum()))

Final Balance: R$ 1262874.33


### Question 07

In the eye of the storm of the global COVID-19 pandemic, the future interest rates skyrocketed! In that manner, the hedge received a bunch of transfers to compensate that surge.

Inversely, the portfolio value plummeted, as interest rates soared. 

Finally, let's check the "hedge efficiency"!

In [21]:
working_days = 1209 - len(DI_future)
coupon_days = list(np.array([77,204,328,455,580,706])- len(DI_future))

ltn_yield_start = 0.06143
rate_start = 0.0565 #(5,65% a.a.)

ltn_yield_end = 0.076
rate_end = 0.076

print("Expected Portfolio Delta: R$ %.2f"%(((float(LTN_price(ltn_yield_end,working_days)) + float(NTNF_price(rate_end,coupon_days))) - ( float(LTN_price(ltn_yield_start,working_days)) + float(NTNF_price(rate_start,coupon_days)))) * quantity_bonds))

Expected Portfolio Delta: R$ -967626.11


As it can be seen, it happened an "overhedge", when considering that both bonds adapted their internal rate of discount acording to the market.

However, taking into consideration the higly volatile interest rates market of that period, it was an acceptable outcome!

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6d35c4cd-e39b-42df-813c-f52fc8d85188' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>